In [ ]:
#https://wikidocs.net/44249 에서 네이버 영화리뷰를 기반으로 한 감성분석방법을 가져왔습니다.

In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 30.5 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
     |████████████████████████████████| 448 kB 37.9 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
import time
from tqdm import tqdm
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#네이버영화리뷰데이터로 모델생성

In [19]:
#데이터 로드
train_data = pd.read_table('/content/drive/MyDrive/Colab Notebooks/ratings_train_all.txt')
train_data = train_data.sample(frac=0.1, random_state=42) ######일부만해봄

#트레인테스트 스플릿
from sklearn.model_selection import train_test_split
X_train, X_test, = train_test_split(train_data, test_size=0.25, random_state=42)
train_data = X_train
test_data = X_test
print('훈련용 리뷰 개수 :',len(train_data))
print('훈련용 리뷰 개수 :',len(test_data))

# document 열에서 중복인 내용이 있다면 중복 제거
train_data.drop_duplicates(subset=['document'], inplace=True) 
print()
print('중복제거 후 트레인셋 수 :',len(train_data))

#결측치 제거
if train_data.isnull().values.any() ==True:
    train_data = train_data.dropna(how = 'any')
    print()
    print('결측치 제거 후 갯수 : ',len(train_data))

# 트레인셋의 라벨 수
print()
print('트레인셋 라벨 수')
print(train_data.groupby('label').size().reset_index(name = 'count'))

#한글 정규화 - 한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
print()
print('한글정규화')
train_data[:5]

훈련용 리뷰 개수 : 15000
훈련용 리뷰 개수 : 5000

중복제거 후 트레인셋 수 : 14847

결측치 제거 후 갯수 :  14846

트레인셋 라벨 수
   label  count
0      0   7390
1      1   7456

한글정규화


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,document,label
93453,7550389,보다가 중간에 나왔음,0
97061,10129143,안보신분들은 무조건 보시라고 추천하고 싶습니다 영화에 대해 별 사전지식 없이 봤는데...,1
141690,2845189,,0
180651,1135035,여자배우가아깝다남자배우랑도 넘안어울려,0
42888,7470122,꿈과 현실 사이의 괴리에 좌절하는 청춘들의 모습을 담백하게 담아냈다 야구를 계속하는...,1


In [22]:
print('다시 생긴 공백이 있다면 삭제')
train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how = 'any')
print('학습데이터 길이 : ' + str(len(train_data)))

#위와 마찬가지로 테스트셋도 전처리
test_data.drop_duplicates(subset = ['document'], inplace=True) # best_comment 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print()
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

#불용어 지정 및 형태소 분리
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

#형태소분리
okt = Okt()

#트레인셋 형태소분리
X_train = []
for sentence in train_data['document']:
    temp_X = okt.morphs(sentence, stem=True) # 형태소분리
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

#테스트셋 형태소분리
X_test = []
for sentence in test_data['document']:
  temp_X = okt.morphs(sentence, stem=True) # 형태소분리
  temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
  X_test.append(temp_X)

다시 생긴 공백이 있다면 삭제
학습데이터 길이 : 14746

전처리 후 테스트용 샘플의 개수 : 4934


In [23]:
#정수인코딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(str(X_train))

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print()
print('단어 집합의 크기 :',vocab_size)

tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

print()
print('정수인코딩 확인', X_train[:3])

y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

print('빈도수 적은 단어들 삭제해서 빈 샘플들이 생겼음. 이를 다시 제거')
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

print()
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))

단어 집합(vocabulary)의 크기 : 1585
등장 빈도가 2번 이하인 희귀 단어의 수: 369
단어 집합에서 희귀 단어의 비율: 23.2807570977918
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 0.07178597393710938

단어 집합의 크기 : 1217

정수인코딩 확인 [[2, 230, 26], [821, 950, 1050, 124, 242, 72, 80, 1, 449, 132, 175, 2, 375, 287, 10, 2, 25, 17, 13, 205, 16], [140, 50, 48, 194, 122, 540]]
14518
14518


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [24]:
#지정한 길이 이상길이 샘플 삭제
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

max_len = 30  #길이지정
below_threshold_len(max_len, X_train)

전체 샘플 중 길이가 30 이하인 샘플의 비율: 97.9473756715801


In [25]:
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

lstm으로 네이버 영화 리뷰 감성 분류하기

In [26]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

#LSTM설정
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

#학습시작
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

Epoch 1/15
194/194 [==============================] - 12s 18ms/step - loss: 0.5023 - acc: 0.7537 - val_loss: 0.4394 - val_acc: 0.7999

Epoch 00001: val_acc improved from -inf to 0.79993, saving model to best_model.h5
Epoch 2/15
194/194 [==============================] - 3s 14ms/step - loss: 0.3903 - acc: 0.8235 - val_loss: 0.4198 - val_acc: 0.8068

Epoch 00002: val_acc improved from 0.79993 to 0.80682, saving model to best_model.h5
Epoch 3/15
194/194 [==============================] - 3s 14ms/step - loss: 0.3691 - acc: 0.8333 - val_loss: 0.4194 - val_acc: 0.8082

Epoch 00003: val_acc improved from 0.80682 to 0.80820, saving model to best_model.h5
Epoch 4/15
194/194 [==============================] - 3s 14ms/step - loss: 0.3565 - acc: 0.8423 - val_loss: 0.4163 - val_acc: 0.8051

Epoch 00004: val_acc did not improve from 0.80820
Epoch 5/15
194/194 [==============================] - 3s 14ms/step - loss: 0.3472 - acc: 0.8467 - val_loss: 0.4165 - val_acc: 0.8058

Epoch 00005: val_acc did no

#네이버영화리뷰데이터로 네이버웹툰댓글 감성점수 매기기

In [27]:
#파일 넣으면 알아서 해주는 모델 만들기
def sentiment_predict(new_sentence):
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = 1500) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    return score

In [49]:
#웹툰댓글파일 불러오기. 한글 43만여 댓글중 라벨링한 데이터 재외한 33만여 댓글들 분석
test_data = pd.read_csv('/content/drive/Shareddrives/데청캠/웹툰 세계화/데이터 모음집/FoDoDic_kw_진짜_최초라벨링.csv')
webtoon_comment = test_data[   ((test_data['pos'] ==0) & (test_data['neg'] ==0)) | 
                               ((test_data['pos'] ==1) & (test_data['neg'] ==-1))       ]

#점수매기기
pred_label=[]
for sentence in tqdm(webtoon_comment['best_comment']):
    try:
        pred_label.append(sentiment_predict(sentence)) 
    except :
        pred_label.append(2)

df = pd.DataFrame({'number' :webtoon_comment['number'],
                   'LabelfromNaverMovieReview' : pred_label,
                   'comment' : webtoon_comment['best_comment']} )
df

,number,best_comment,pos,neg
0,0,BEST-신의탑을 않본사람은 있어도 한번본 사람은 없다-,0,0
1,1,"BEST우리나라에서 유일하게 원피스,나루토,진격의거인이랑 맞먹는 웹툰이다ㅋㅋ",0,0
2,2,BESTSIU가 달려가면서 그림그리기를 자유자재로 구사하는것은 기본이고 선긋기같은 ...,0,0
3,3,BEST월요일을 위로 받는 방법,0,0
4,4,BEST탑에 꼭대기엔 작가님이 계시다,0,0
...,...,...,...,...
434555,434555,BEST바름유연 둘이 벌써 3주년이구나... 이제 슬슬 결혼하자,0,0
434563,434563,"BEST루리웹, 클리앙, 보배드림, 축리웹 댓글테러단들 지금까지 뭐했어?맨날 열등감...",0,0
434566,434566,BEST작가님 지켜드리지 못해 죄송합니다....다들 좀 조용히 했으면 좋겠는데 말이...,0,0
434578,434578,"BEST루리웹, 클리앙, 보배드림, 축리웹 댓글테러단들 지금까지 뭐했어?맨날 열등감...",0,0


#포도사전 - 위와 같은 방식으로, 직접라벨링한 댓글10만개를 토대로 나머지 댓글 분석

In [ ]:
#데이터 로드
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FoDoDic_kw_진짜_최초라벨링.csv')

#필요한 칼럼만 부르고, 긍정이거나 부정으로 분류된 댓글만 뽑기
train_data = train_data[['number','best_comment','pos','neg']]
train_data = train_data[(train_data['pos']==1) | (train_data['neg']==-1)]
train_data

#트레인 테스트셋 분리
from sklearn.model_selection import train_test_split
X_train, X_test, = train_test_split(train_data, test_size=0.25, random_state=42)
train_data = X_train
test_data = X_test

#중복 데이터 수
print(train_data['best_comment'].nunique(), train_data['pos'].nunique())

# 중복 제거
train_data.drop_duplicates(subset=['best_comment'], inplace=True) 

#결측치있다면 삭제
if train_data.isnull().values.any() ==True:
    train_data = train_data.dropna(how = 'any')
    print('결측치 제거 후 갯수')
    len(train_data)

#불용어지정
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

#형태소분리
okt = Okt()

#위의 과정을 테스트셋에도 
test_data['best_comment'].nunique(), test_data['pos'].nunique()
test_data.drop_duplicates(subset=['best_comment'], inplace=True) 
if test_data.isnull().values.any() ==True:
    test_data = test_data.dropna(how = 'any')
    print('결측치 제거 후 갯수')
    len(test_data)
test_data

X_train = []
for sentence in train_data['best_comment']:
    temp_X = okt.morphs(sentence, stem=True) # 형태소분리
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

X_test = []
for sentence in test_data['best_comment']:
  temp_X = okt.morphs(sentence, stem=True) # 형태소분리
  temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
  X_test.append(temp_X)

#정수인코딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(str(X_train))

tokenizer.fit_on_texts(str(X_test))

print(tokenizer.word_index)

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

vocab_size = total_cnt - rare_cnt + 1

tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(X_test)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

y_train = np.array(train_data['pos'])
y_test = np.array(test_data['pos'])

# 빈도수 적은 단어들 삭제해서 다시 생긴 빈 샘플들 삭제
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

#아래 길이 이상의 댓글 삭제
max_len = 60
below_threshold_len(max_len, X_train)

X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))


es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='acc', mode='max', verbose=1, save_best_only=True)


model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

In [ ]:
#웹툰댓글파일 불러오기. 한글 43만여 댓글중 라벨링한 데이터 재외한 33만여 댓글들 분석
test_data = pd.read_csv('/content/drive/Shareddrives/데청캠/웹툰 세계화/데이터 모음집/FoDoDic_kw_진짜_최초라벨링.csv')
webtoon_comment = test_data[   ((test_data['pos'] ==0) & (test_data['neg'] ==0)) | 
                               ((test_data['pos'] ==1) & (test_data['neg'] ==-1))       ]

#점수매기기
pred_label=[]
for sentence in tqdm(webtoon_comment['best_comment']):
    try:
        pred_label.append(sentiment_predict(sentence)) 
    except :
        pred_label.append(2)

df = pd.DataFrame({'number' :webtoon_comment['number'],
                   'LabelfromNaverMovieReview' : pred_label,
                   'comment' : webtoon_comment['best_comment']} )
df